# Aplicando modelagem de assuntos ao DHBB
Neste capítulo vamos explorar ferramentas de modelagem de assuntos e explorar aplicações ao DHBB. Como sempre, começamos com alguns imports familiares.

In [33]:
import os, glob, pickle
import spacy
from spacy import displacy
from sqlalchemy import create_engine
from dhbbmining import *
import ipywidgets as widgets
from tqdm import tqdm

import pandas as pd
import time
import re
import json
import os
from pprint import pprint
from IPython.display import clear_output

import string

Vamos também carregar o modelo de NLP para a língua portuguesa do Spacy:

In [3]:
nlp = spacy.load("pt_core_news_sm")

Agora faremos alguns imports novos, particularmente da biblioteca [Gensim](https://radimrehurek.com/gensim), que nos oferece as ferramentas que necessitamos para modelagem de assuntos.

In [4]:
from string import punctuation
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec, word2vec

Para minimizar o uso de memória, vamos construir uma classe para representar o nosso corpus como um iterador, operando diretamente do banco de dados. Desta forma, ao fazer nossas análises, podemos carregar um documento por vez para alimentar os modelos, sem a necessidade de manter todo o corpus na memória, economizando memória RAM.

# retrieve data from MongoDB

In [14]:
import pymongo
from pymongo import MongoClient
import dns
client = MongoClient()

In [15]:
db = client.twitter
collection_ufmg = db.ufmg_filtered

In [31]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [34]:
data['tokens'] = data['text'].apply(lambda x: tknzr.tokenize(x.translate(str.maketrans('', '', string.punctuation)).lower()))

In [38]:
VT = list(data['tokens'])

# continue

Abaixo um pequeno exemplo de como a classe `DHBBCorpus` funciona:

In [13]:
DC = DHBBCorpus(5)
count = 0
for f in DC:
    count +=1
    #pass
    print(f)
    if count == 10: break

['«José', 'Machado', 'Coelho', 'de', 'Castro»', 'nasceu', 'em', 'Lorena', 'SP']
['Estudou', 'no']
['Ginásio', 'Diocesano', 'de', 'São', 'Paulo', 'e', 'bacharelou-se', 'em', '1910']
['pela', 'Faculdade', 'de', 'Ciências', 'Jurídicas', 'e', 'Sociais']
['Dedicando-se']
['à', 'advocacia']
['foi', 'promotor', 'público', 'em', 'Cunha', 'SP', 'e', 'depois', 'delegado', 'de', 'polícia', 'no']
['Rio', 'de', 'Janeiro', 'então', 'Distrito', 'Federal']
['Iniciou', 'sua', 'vida', 'política', 'como', 'deputado', 'federal', 'pelo', 'Distrito', 'Federal', 'exercendo', 'o', 'mandato', 'de', '1927', 'a', '1929']
['Reeleito', 'para', 'a', 'legislatura', 'iniciada', 'em', 'maio', 'de', '1930', 'ocupava', 'sua', 'cadeira', 'na', 'Câmara', 'quando', 'em', '3', 'de', 'outubro', 'foi', 'deflagrado', 'o', 'movimento', 'revolucionário', 'liderado', 'por', 'Getúlio', 'Vargas']


## Word2vec 
Vamos começar pelo treinamento de um modelo word2vec. Este modelo itera 6 vezez sobre o corpus logo, devemos ver o contador atingir 46122. Estas repetições são necessárias para permitir a 

In [39]:
if os.path.exists('virus_tweets.w2v'):
    model = Word2Vec.load('virus_tweets.w2v')
else:
    #DC = DHBBCorpus()
    model = Word2Vec(sentences=VT, workers=32)
    model.save('virus_tweets.w2v')

### Explorando o modelo

In [40]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

musica
dos
vizinhos
zika
aq
da
comunidade
sei
que
estou


In [42]:
list(model.wv['deputado'])

[2.3886912,
 0.28217658,
 -1.9874773,
 -0.51558894,
 2.1879008,
 -0.19042288,
 0.6908539,
 -1.0632348,
 -0.67359257,
 3.0175622,
 2.2130032,
 -1.4433424,
 2.760831,
 -2.1444695,
 -0.2821184,
 0.58656615,
 1.0061878,
 1.1807309,
 0.54593456,
 -2.3770607,
 0.43160978,
 0.050366312,
 0.8009387,
 -1.0630683,
 -0.29855117,
 1.0814375,
 0.6338707,
 -0.58640933,
 -3.4926078,
 2.4008555,
 -0.5238663,
 -0.7037424,
 0.7415925,
 -0.30719492,
 0.9929472,
 5.1566324,
 0.24720924,
 -3.5077336,
 -2.0185099,
 -1.5160787,
 -0.6066206,
 1.5647931,
 2.6941617,
 -2.4217505,
 3.4705436,
 -0.01701793,
 0.87891155,
 1.8238404,
 -1.3465565,
 -1.3153664,
 2.1167681,
 -0.6209916,
 -3.9595716,
 -1.1521416,
 -2.2909615,
 -3.2971458,
 2.292861,
 0.5738925,
 -2.6859908,
 -0.5351569,
 2.9647572,
 -0.8235569,
 1.3786242,
 -2.7816575,
 1.4543685,
 -2.0188134,
 1.541914,
 -1.7891498,
 0.09042452,
 1.3875531,
 3.1705196,
 1.4111038,
 0.6391565,
 1.4406295,
 4.193468,
 -0.047331657,
 -2.1729064,
 -0.7105255,
 0.08725463,

In [43]:
len(model.wv.vocab)

179911

In [46]:
model.wv.most_similar(positive=['dengue'], topn=20)

[('aedes', 0.6367322206497192),
 ('denguezika', 0.6295568943023682),
 ('de', 0.6234614849090576),
 ('virus', 0.609860360622406),
 ('zika', 0.5731625556945801),
 ('que', 0.5616250038146973),
 ('a', 0.5592842102050781),
 ('rt', 0.5592179298400879),
 ('e', 0.5446224212646484),
 ('o', 0.5232881307601929),
 ('zikadengue', 0.5191036462783813),
 ('no', 0.5125505924224854),
 ('nao', 0.5100154876708984),
 ('doenca', 0.5021892189979553),
 ('se', 0.49738809466362),
 ('ja', 0.4926435947418213),
 ('mas', 0.48960384726524353),
 ('aegypti', 0.47452616691589355),
 ('httpstcobvxuoriibu', 0.4638242721557617),
 ('la', 0.4629664421081543)]

In [47]:
model.corpus_total_words

60306788